In [1]:
import cv2
import numpy as np

In [2]:
path1 = r'imgs\IMG_20240510_172748.jpg'
path2 = r'imgs\IMG_20240510_172837.jpg'
path3 = r'imgs\IMG_20240510_172930.jpg'

In [17]:
def getCountors(frame, draw = True):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frame_gray = cv2.GaussianBlur(frame_gray, ksize = (5, 5), sigmaX=1)

    frame_gray = cv2.Canny(frame_gray, threshold1=50, threshold2=125)

    cntrs, h = cv2.findContours(frame_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = []

    for cnt in cntrs:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*perimeter, closed = True)
        bbox = cv2.boundingRect(approx)

        cnts.append((cnt, approx, area, perimeter, bbox))

    cnts = sorted(cnts, key = lambda x: -x[2])
    
    cnt = [c[0] for c in cnts[:1]]

    if draw:
        M = cv2.moments(cnt[0])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        # Draw the center of the contour
        cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)

        frame = cv2.drawContours(frame, cnt, -1, (0, 0, 255), 5)

    cnt = cnt[0]
    l = tuple(cnt[cnt[:,:,0].argmin()][0])
    r = tuple(cnt[cnt[:,:,0].argmax()][0])
    t = tuple(cnt[cnt[:,:,1].argmin()][0])
    b = tuple(cnt[cnt[:,:,1].argmax()][0])


    return frame, cnts, (l, r, t, b)

img = cv2.imread('imgs\IMG_20240510_172930.jpg')

w, h = img.shape[1]//4, img.shape[0]//4

img = cv2.resize(img, (w, h))

img_cnts, cnts1, (l, r, t, b) = getCountors(img)


cv2.imshow('Detected Circles', img_cnts)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
cv2.circle(img_cnts, l, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, r, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, t, 8, (0, 255, 0), -1)
cv2.circle(img_cnts, b, 8, (0, 255, 0), -1)

p1 = (l[0], t[1])
p2 = (r[0], t[1])
p3 = (l[0], b[1])
p4 = (r[0], b[1])

cv2.circle(img_cnts, p1, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p2, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p3, 8, (255, 255, 0), -1)
cv2.circle(img_cnts, p4, 8, (255, 255, 0), -1)

# Display the image
cv2.imshow('Extreme Points', img_cnts)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
(l, r, t, b)

((274, 440), (970, 466), (634, 113), (612, 795))

In [4]:
img = cv2.imread(path2)

w, h = img.shape[1]//4, img.shape[0]//4

img = cv2.resize(img, (w, h))

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# gray_blurred = cv2.medianBlur(gray_img, 21, 1)
gray_blurred = cv2.blur(gray_img, (7, 7), 1)
# gray_blurred = cv2.GaussianBlur(gray_img, (21, 21), 7)

detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20,  
                                   param1=50, param2=175, minRadius=0, maxRadius=0)

# detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20, 
#                                    param1=50, param2=220, minRadius=0, maxRadius=0)

if detected_circles is not None:
    detected_circles = np.uint16(np.around(detected_circles))
    circles = detected_circles[0, :]
    circles = sorted(circles, key=lambda x: x[2], reverse=True)
    
    # Draw only the 5 largest circles
    i = 0
    for circle in circles:
        (x, y, r) = circle
        cv2.circle(img, (x, y), r, (0, 255, 0), 2)
        cv2.circle(img, (x, y), 1, (0, 0, 255), 3)
        if i == 5: break


cv2.imshow('Detected Circles', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
cv2.imshow('Detected Circles', gray_blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
(21, 21), 7

In [71]:
import cv2
import numpy as np

# Load the image
img = cv2.imread(path1)

img = cv2.resize(img, (img.shape[1]//4, img.shape[0]//4))

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply thresholding to segment the objects
_, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on the original image
for contour in contours:
    cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)

# Display the output
cv2.imshow('Contours', img)
cv2.waitKey(0)
cv2.destroyAllWindows()